In [88]:
import pandas as pd

In [89]:
df_thread = pd.read_csv('thread_18cff0b946f3a4d4_cleaned.csv', index_col='Unnamed: 0')
df_thread.info()

<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 76 to 100
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   threadId          11 non-null     object
 1   from              11 non-null     object
 2   date              11 non-null     object
 3   body_cleaned      11 non-null     object
 4   body_cleaned_len  11 non-null     int64 
dtypes: int64(1), object(4)
memory usage: 528.0+ bytes


In [90]:
df_thread = df_thread.reset_index().drop('index', axis=1)
df_thread

,threadId,from,date,body_cleaned,body_cleaned_len
0,18cff0b946f3a4d4,Felipe dos Santos Giacomel <felipe.giacomel@tr...,2024-01-12 15:59:44-03:00,"Prezada DGBD, boa tarde!\r\n\r\nEstamos com er...",105
1,18cff0b946f3a4d4,Edson Elnei Lehr <elehr@trt4.jus.br>,2024-01-12 16:29:16-03:00,Copiando para o e-mail correto da DIBD.\r\n\r\...,15
2,18cff0b946f3a4d4,Edson Elnei Lehr <elehr@trt4.jus.br>,2024-01-12 17:09:03-03:00,Parece ser de tnsnames.ora desatualizado na es...,36
3,18cff0b946f3a4d4,Felipe dos Santos Giacomel <felipe.giacomel@tr...,2024-01-12 17:29:30-03:00,"Obrigado, Edson.",2
4,18cff0b946f3a4d4,Evandro Bassanesi <evandro.bassanesi@trt4.jus.br>,2024-01-12 17:34:50-03:00,"Felipe,\r\n\r\nEste está desatualizado. O banc...",24
5,18cff0b946f3a4d4,Felipe dos Santos Giacomel <felipe.giacomel@tr...,2024-01-12 17:46:45-03:00,Vamos atualizar. Obrigado!\r\n\r\n\r\nAtencios...,18
6,18cff0b946f3a4d4,Paulo Ricardo Ritter <pritter@trt4.jus.br>,2024-01-15 08:58:54-03:00,"Felipe, isso não ocorre no servidor do Markvis...",104
7,18cff0b946f3a4d4,Paulo Ricardo Ritter <pritter@trt4.jus.br>,2024-01-15 09:19:39-03:00,"Agora que entendi, o problema que refere, não...",46
8,18cff0b946f3a4d4,Paulo Ricardo Ritter <pritter@trt4.jus.br>,2024-01-15 10:18:12-03:00,A GPO de atualização do TNSnames (desktop-all-...,28
9,18cff0b946f3a4d4,Francisco Flesch <francisco.flesch@trt4.jus.br>,2024-01-15 11:05:28-03:00,Eu troquei o TNSnames na última vez que deu pr...,41


In [91]:
from langchain_openai import ChatOpenAI, OpenAI

# llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

llm_json = ChatOpenAI(model="gpt-4o-mini").bind(
    response_format={"type": "json_object"}
)

In [97]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=0,
    length_function=len,
)

In [98]:
df_thread['splits'] = df_thread.apply(lambda row: splitter.split_text(row['body_cleaned']), axis=1)
df_thread['splits']

0     [Prezada DGBD, boa tarde!, Estamos com erro ao...
1     [Copiando para o e-mail correto da DIBD.\r\n\r...
2     [Parece ser de tnsnames.ora desatualizado na e...
3                                    [Obrigado, Edson.]
4     [Felipe,\r\n\r\nEste está desatualizado. O ban...
5     [Vamos atualizar. Obrigado!\r\n\r\n\r\nAtencio...
6     [Felipe, isso não ocorre no servidor do Markvi...
7     [Agora que entendi, o problema  que refere, nã...
8     [A GPO de atualização do TNSnames (desktop-all...
9     [Eu troquei o TNSnames na última vez que deu p...
10    [Obrigado, Paulo!\r\n\r\nSe não houver objeção...
Name: splits, dtype: object

In [95]:
prompt_grader_importancy_system = """
    You are a helpful assistant who is in charge of classifying strings as either interesting to a use case or not.
    This strings will be used by another llm to generate a report and a knowledge base fo a IT company.
    So the use case is to determine if the string contain helpful information to create a knowledge base or not.
    Helpefull information (grade as True): problem, information about something, solutions, questions
    Not helpfull information (grade as False): names, emails, phone numbers, someone saying that wil send email or something like this.

    You should output a json. Key shold be the key of the string, the value should be the boolean value
"""


prompt_grader_importancy_user = """
    {STRINGS}
"""

from langchain_core.prompts import ChatPromptTemplate

prompt_grader = ChatPromptTemplate.from_messages([
    ("system", prompt_grader_importancy_system), 
    ("user", prompt_grader_importancy_user)
    ])

In [96]:
# chain_grader = prompt_grader | llm
chain_grader = prompt_grader | llm_json

In [101]:
def string_grader(row):
    if len(row):
        strings = { k:v for (k,v) in zip(range(len(row)), row) }

        # remove all uninformative strings
        response = chain_grader.invoke( { 'STRINGS': strings } )
        strings_graded = response.content

        strings_graded = json.loads(strings_graded)

        # # get useful strings
        strings_graded_filtered = [ strings[int(k)] for (k,v) in strings_graded.items() if v == True ]
        
        return "\n\n".join(strings_graded_filtered)

In [102]:
df_thread['splits_selected'] = df_thread.apply(lambda row: string_grader(row['splits']), axis=1)
df_thread['splits_selected']

0     Estamos com erro ao fazer operações na base do...
1                                                      
2     Parece ser de tnsnames.ora desatualizado na es...
3                                                      
4     Felipe,\r\n\r\nEste está desatualizado. O banc...
5                                                      
6     TNSnames  fica no servidor  Heitor, e  todas a...
7     Agora que entendi, o problema  que refere, não...
8     A GPO de atualização do TNSnames (desktop-all-...
9     Eu troquei o TNSnames na última vez que deu pr...
10    Obrigado, Paulo!\r\n\r\nSe não houver objeção ...
Name: splits_selected, dtype: object

In [105]:
df_thread['len_count'] = df_thread.apply(lambda row: len(str(row['splits_selected']).split()), axis=1)

In [119]:
df_thread.to_csv("thread.csv", index=False)

In [109]:
prompt_report_creator_system = """
    You are an expert IT technical writer that create draft of knowledge base documents.
    You dont need to give any IT support, you should be impersonal like a newswriter giving a news.
    
    You will receive two inputs: a knowledge base document of a IT company and a list of strings extracted from a conversation between IT Technicians.
    You should consolidate the strings into the knowledge base document.
    If the previous report is empty, just create a new one from scratch.

    The language of the conversation is generaly in Brazilian Portuguese. So your report should be in Brazilian Portuguese.
    
    Dont write uninformative phrases like "relatorio de conversa".
    You should output a technical report structured like a book.    
    Do not discuss the email itself, focus rewriting the content
    Do not sugest any solution or output any information that are not in the strings or in the previous report.
    Dont write like: "O email fala sobre um problema tal e bla bla.". 
    You should be more directly to point.

    You should output a json. 
        Key 'subject': a sentence describing the subject or description of the problem happening. Generaly it is in the first sentences of the conversation because it is the email that start the conversation.
        Key 'technologys': the name of the technologys being addressed.
        Key 'description': consolidate the report with the strings in order in a logical, elegant and techical way. 
                    Convert first person sentences to third person.
                    Some strings should by copied char by char, like log strings, error messages.
                    In the case of instructions with steps on how to do something you should rewrite it. Dont loose anyinformation. 
                    If the case exists a error message, write char by char of the error message that happened. you should just copy system messages.
                    You could create/use sections like a book.
        Key 'key_take_aways': Write a bullet list of the most important configurations mentioned in the conversation. Maybe there is something that someone forgot to do or said that is needed to do.
        Key 'solution': If there is a solution mentioned in the conversation or in te previous report, write a summary of the solution found in the conversation. 
                    If dont have any solution mentioned, write 'Não há menção de nenhuma solução'.
                    Your task is not to suggest any solution, your task is to report what is in the report and in the conversation.
    """

prompt_report_creator_user = """
    {STRINGS}
"""

from langchain_core.prompts import ChatPromptTemplate

prompt_report_creator = ChatPromptTemplate.from_messages(
    [("system", prompt_report_creator_system), ("user", prompt_report_creator_user)]
)

In [110]:
chain_report_creator = prompt_report_creator | llm_json

In [113]:
import json

response = chain_report_creator.invoke( { 'STRINGS': '\n\n'.join( df_thread['splits_selected'].to_list() ) } )
new_report = json.loads(response.content)

pretty_json = json.dumps(new_report, indent=4)
print(pretty_json)

{
    "subject": "Erro ao fazer opera\u00e7\u00f5es na base do Infor de S\u00e3o Leopoldo.",
    "technologys": [
        "Infor",
        "VMware",
        "TNSnames",
        "GPO"
    ],
    "description": "Um erro foi relatado ao tentar realizar opera\u00e7\u00f5es na base do Infor de S\u00e3o Leopoldo. O log de erro gerado foi:\n\n[2024-01-12 11:41:18] Falha ao conectar \u2026 base do inFOR de S\u00c6o Leopoldo 1\u00a6\n\n> VT (SAOLEO.WORLD) com o usu\u00e1rio inforadm: Exception calling \"Open\" with \"0\".\n\nFoi observado que o problema pode estar relacionado a um arquivo tnsnames.ora desatualizado na esta\u00e7\u00e3o de trabalho onde o erro ocorreu. O banco de dados foi acessado normalmente e foi confirmado que o listener est\u00e1 ativo. \n\nFoi destacado que o banco de S\u00e3o Leopoldo foi migrado para VMware e que o arquivo TNSnames deve ser atualizado com a vers\u00e3o mais recente dispon\u00edvel na rede. O arquivo TNSnames est\u00e1 localizado no servidor Heitor, e tod

In [118]:
import textwrap

wrapped_description = textwrap.fill(new_report['description'], width=155)
print(wrapped_description)

Um erro foi relatado ao tentar realizar operações na base do Infor de São Leopoldo. O log de erro gerado foi:  [2024-01-12 11:41:18] Falha ao conectar …
base do inFOR de SÆo Leopoldo 1¦  > VT (SAOLEO.WORLD) com o usuário inforadm: Exception calling "Open" with "0".  Foi observado que o problema pode estar
relacionado a um arquivo tnsnames.ora desatualizado na estação de trabalho onde o erro ocorreu. O banco de dados foi acessado normalmente e foi confirmado
que o listener está ativo.   Foi destacado que o banco de São Leopoldo foi migrado para VMware e que o arquivo TNSnames deve ser atualizado com a versão
mais recente disponível na rede. O arquivo TNSnames está localizado no servidor Heitor, e todas as estações do TRT têm acesso de leitura a ele. A GPO é
responsável pela cópia do arquivo do servidor Heitor para as estações.  Foi questionado se existe algum chamado relacionado ao problema ou se há estações
com problemas no Infor. A análise indicou que o problema não está na localidade

In [120]:
prompt_report_creator_system = """
    You are an expert IT technical writer that create draft of knowledge base documents.
    You dont need to give any IT support, you should be impersonal like a newswriter giving a news.
    
    You will receive a list of strings extracted from a conversation between IT Technicians.
    The language of the conversation is generaly in Brazilian Portuguese. So your report should be in Brazilian Portuguese.
    You should output a technical report structured like a book.
    The idea here is to identify pontential information that should be documented in a new knowledge base document. For example, somthing that someone forgot to do. Technologies configurations or infrastructure.
    
    Dont write uninformative phrases like "relatorio de conversa".
    Do not discuss the email itself, focus rewriting the content
    Do not sugest any solution or output any information that are not in the strings or in the previous report.
    Dont write like: "O email fala sobre um problema tal e bla bla.". 
    You should be more directly to point.

    You should output a json. 
        Key 'key_take_aways': Write a bullet list of the most important configurations mentioned in the conversation. 
                                Maybe there is something that someone forgot to do or said that is needed to do.
                                You should put here what new knowledge you could infer from the conversation.
        
    """

prompt_report_creator_user = """
    {STRINGS}
"""

from langchain_core.prompts import ChatPromptTemplate

prompt_report_creator = ChatPromptTemplate.from_messages(
    [("system", prompt_report_creator_system), ("user", prompt_report_creator_user)]
)

In [121]:
chain_report_creator = prompt_report_creator | llm_json

In [122]:
import json

response = chain_report_creator.invoke( { 'STRINGS': '\n\n'.join( df_thread['splits_selected'].to_list() ) } )
new_report = json.loads(response.content)

pretty_json = json.dumps(new_report, indent=4)
print(pretty_json)

{
    "key_take_aways": [
        "Erro ao conectar na base do Infor de S\u00e3o Leopoldo identificado, possivelmente relacionado ao arquivo tnsnames.ora desatualizado.",
        "Banco de dados de S\u00e3o Leopoldo foi migrado para VMware.",
        "O TNSnames est\u00e1 localizado no servidor Heitor e todas as esta\u00e7\u00f5es do TRT t\u00eam acesso de leitura a ele.",
        "A GPO \u00e9 respons\u00e1vel pela c\u00f3pia do tnsnames.ora do servidor Heitor para as esta\u00e7\u00f5es, mas a GPO de atualiza\u00e7\u00e3o do TNSnames (desktop-all-oracle) n\u00e3o havia rodado.",
        "Foi necess\u00e1rio atualizar manualmente o TNSnames na esta\u00e7\u00e3o onde ocorreu o erro devido \u00e0 falha da GPO.",
        "Sugest\u00e3o de altera\u00e7\u00e3o da GPO para ser aplicada no contexto *trt4.gov.br/infraestrutura/DIM/MARKVISION*."
    ]
}


In [123]:
import textwrap

wrapped_description = textwrap.fill(new_report['description'], width=155)
print(wrapped_description)

KeyError: 'description'

In [124]:
prompt_report_creator_system = """
    You are an expert IT technical writer that create draft of knowledge base documents.
    You dont need to give any IT support, you should be impersonal like a newswriter giving a news.
    
    You will receive a list of strings extracted from a conversation between IT Technicians.
    The language of the conversation is generaly in Brazilian Portuguese. So your report should be in Brazilian Portuguese.
    You should output a technical report structured like a book.
    The idea here is to identify pontential information that should be documented in a new knowledge base document. For example, somthing that someone forgot to do. Technologies configurations or infrastructure.
    
    Dont write uninformative phrases like "relatorio de conversa".
    Do not discuss the email itself, focus rewriting the content
    Do not sugest any solution or output any information that are not in the strings or in the previous report.
    Dont write like: "O email fala sobre um problema tal e bla bla.". 
    You should be more directly to point.

    You should output a json. 
        Key 'subject': a sentence describing the subject or description of the problem happening. Generaly it is in the first sentences of the conversation because it is the email that start the conversation.
        Key 'technologys': the name of the technologys being addressed.
        Key 'description': consolidate the report with the strings in order in a logical, elegant and techical way. 
                    Convert first person sentences to third person.
                    Some strings should by copied char by char, like log strings, error messages.
                    In the case of instructions with steps on how to do something you should rewrite it. Dont loose anyinformation. 
                    If the case exists a error message, write char by char of the error message that happened. you should just copy system messages.
                    You could create/use sections like a book.
        Key 'key_take_aways': Write a bullet list of the most important configurations mentioned in the conversation. 
                                Maybe there is something that someone forgot to do or said that is needed to do.
                                You should put here what new knowledge you could infer from the conversation.
    """

prompt_report_creator_user = """
    {STRINGS}
"""

from langchain_core.prompts import ChatPromptTemplate

prompt_report_creator = ChatPromptTemplate.from_messages(
    [("system", prompt_report_creator_system), ("user", prompt_report_creator_user)]
)

In [125]:
chain_report_creator = prompt_report_creator | llm_json

In [126]:
import json

response = chain_report_creator.invoke( { 'STRINGS': '\n\n'.join( df_thread['splits_selected'].to_list() ) } )
new_report = json.loads(response.content)

pretty_json = json.dumps(new_report, indent=4)
print(pretty_json)

{
    "subject": "Erro ao realizar opera\u00e7\u00f5es na base do Infor de S\u00e3o Leopoldo.",
    "technologys": [
        "Infor",
        "VMware",
        "GPO",
        "TNSnames"
    ],
    "description": {
        "Introdu\u00e7\u00e3o": "Uma opera\u00e7\u00e3o na base de dados do Infor em S\u00e3o Leopoldo apresentou um erro.",
        "Erro Reportado": "[2024-01-12 11:41:18] Falha ao conectar \u2026 base do inFOR de S\u00c6o Leopoldo 1\u00a6> VT (SAOLEO.WORLD) com o usu\u00e1rio inforadm: Exception calling \"Open\" with \"0\".",
        "Diagn\u00f3stico Inicial": "Foi identificado que o erro pode ser decorrente de um arquivo tnsnames.ora desatualizado na esta\u00e7\u00e3o de trabalho onde o erro ocorreu. Foi mencionado que o banco de dados foi migrado para VMware e que o listener estava ativo.",
        "Atualiza\u00e7\u00e3o do TNSnames": "Foi sugerido que o arquivo TNSnames fosse atualizado com a vers\u00e3o mais recente dispon\u00edvel na rede. O arquivo TNSnames encontra

In [127]:
import textwrap

wrapped_description = textwrap.fill(new_report['description'], width=155)
print(wrapped_description)

AttributeError: 'dict' object has no attribute 'expandtabs'